In [1]:
#@title Librerias
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.integrate import quad, fixed_quad, trapezoid, simpson
from sympy import symbols, diff, integrate, lambdify

# Parcial II - Métodos Computacionales (Integrales y Matrices)
**2024-2**

---

*Nombre:* Kevin Cortés Cordero

*Identificación:* 1032010342


---
* Sea honest@
* El uso de herramientas de IA no está permitido
* Puede consultar la documentación de las librerias.
* Sea ordendado dando la solución a cada punto.
* *Asegurese que el notebook corra completamente*
* Suba su notebook al GitHub (en su carpeta) y recuerde hacer pull-request

# 1

**(20 puntos)**
Considera la siguiente integral:

$$
I = \int_0^1 \int_{x^2}^{x+1} \int_{y^2}^{y+2} \int_{z^2}^{z+3} (x+y+z+w) \, dw \, dz \, dy \, dx
$$

Encuentre la solución númerica usando unicamente `quad`. Encuentre el error relativo, para esto emplee el siguiente código que usa `nquad`:


```python
# Definir el integrando
integrand = lambda w, z, y, x: x + y + z + w

# Definir los límites usando funciones lambda
w_limits = lambda z, y, x: [z**2, z+3]
z_limits = lambda y, x: [y**2, y+2]
y_limits = lambda x: [x**2, x+1]
x_limits = lambda: [0, 1]

# Calcular la integral usando nquad
result, error = nquad(
    integrand,
    [w_limits, z_limits, y_limits, x_limits]
)

print(f"Resultado de la integral: {result}")
print(f"Error estimado: {error}")
```



In [126]:
f = lambda x,y,z,w : x+y+z+w

integral_1= quad(lambda z : f(x,y,z,w), a = lambda x,y,z: z**2, b = lambda x,y,z: z+3)


TypeError: '<' not supported between instances of 'function' and 'function'

# 2

**(40 puntos)**
Un lazo de corriente de radio $a$  transporta una corriente $I$ en el punto P que está a una distancia $r$ del centro del lazo con coordenadas esféricas (r, θ, φ). Resuelve para la componente φ del potencial vectorial en el punto $P$ en términos de integrales elípticas:

\begin{equation}
A_{\phi}(r, \theta)=\frac{\mu_0}{4\pi}\frac{4Ia}{\sqrt{a^2+r^2+2ar\sin \theta}}\left(\frac{(2-k^2)K(k)-2E(k)}{k^2} \right)
\end{equation}

donde

\begin{equation}
K(k)=\int_{0}^{\pi/2} \frac{\mathrm{d\phi}}{\sqrt{1-k^2\sin^2\phi}}
\end{equation}

\begin{equation}
E(k)=\int_{0}^{\pi/2} \mathrm{d\phi} \sqrt{1-k^2\sin^2\phi}
\end{equation}

\begin{equation}
k^2=\frac{4ar\sin\theta}{a^2+r^2+2ar\sin\theta}
\end{equation}

Aquí, $K(k)$ es una integral elíptica completa del primer tipo y $E(k)$ es una integral elíptica completa del segundo tipo.


Para
* $a$ = 1,
* $I$ = 3,
* $μ0/4π$ = 1


Usando funciones explicitas explique claramente las entradas, salidas y procedimientos dentro de las mismas. Recuerde para resolver las integrales usar `quad` y grafique para:

*a)* $A_φ(r = 1.1, θ)$ vs. $0\leq θ\leq \pi$

*b)* $A_φ(r, θ = π∕3)$ vs. $0\leq r \leq 10$

Tenga cuidado con la convergencia de su integral



In [129]:
r = 1.1
theta = np.pi

k2 = lambda r, theta : (4*r*np.sin(theta)) / (1 + r**2 + 2*r*np.sin(theta))

integrandoK = lambda k2, phi: 1 / (np.sqrt(1 - k2 * np.sin(phi)**2))


quad(lambda phi: integrandoK(k2(r, theta), phi), 0, np.pi/2-1e-6)

(1.5707953267948966, 1.7439331387812913e-14)

In [130]:
#inciso a

x = 1.1
t = np.linspace(0, np.pi)

k2 = lambda x, t : (4*x*np.sin(t)) / (1 + x**2 + 2*x*np.sin(t))

integrandoK = lambda k2, p: 1 / (np.sqrt(1-k2(x,t) * np.sin(p)**2))
integrandoE = lambda k2, p: (np.sqrt(1-k2(x,t) * np.sin(p)**2))

integralK = quad(lambda p: integrandoK(k2(x,t), p), 0, np.pi)
integralE = quad(lambda p: integrandoE(k2(x,t), p), 0, np.pi)

A = lambda k2, K, E, t: 12 / (np.sqrt(2 + 2*r * np.sin(t))) * (2-k2 * K - 2*E) / (k2)

TypeError: 'numpy.float64' object is not callable

In [131]:
#inciso b

x = np.linspace(0,10,100)
t = np.pi / 3

k2 = lambda x, t : (4*x*np.sin(t)) / (1 + x**2 + 2*x*np.sin(t))

integrandoK = lambda k2, p: 1 / (np.sqrt(1-k2(x,t) * np.sin(p)**2))
integrandoE = lambda k2, p: (np.sqrt(1-k2(x,t) * np.sin(p)**2))

integralK = quad(lambda p: integrandoK(k2(x,t), p), 0, np.pi)
integralE = quad(lambda p: integrandoE(k2(x,t), p), 0, np.pi)

A = lambda k2, K, E, t: 12 / (np.sqrt(2 + 2*r * np.sin(t))) * (2-k2 * K - 2*E) / (k2)

TypeError: 'numpy.ndarray' object is not callable

# 3

(**40 puntos**) El **método de la potencia inversa** es una técnica iterativa utilizada para calcular el autovalor de menor magnitud de una matriz cuadrada. Es una variante del **método de la potencia**, que normalmente encuentra el autovalor de mayor magnitud. Para ello, se aprovecha la propiedad de la matriz inversa:

$$ A^{-1} v = \frac{1}{\lambda} v $$


## Algoritmo

El método de la potencia inversa sigue los siguientes pasos:

1. Elegir un vector inicial $x^{(0)}$ no nulo.

2. Para cada iteración $k$, resolver el sistema lineal:
   
   $$ A y^{(k)} = x^{(k-1)} $$
   
   donde $y^{(k)}$ es un **vector intermedio** que se obtiene resolviendo el sistema lineal. Este vector representa la solución del sistema en cada iteración y es crucial para aproximar el autovector de menor autovalor.

3. Normalizar $y^{(k)}$ para obtener un nuevo vector de iteración:
   
   $$ x^{(k)} = \frac{y^{(k)}}{\| y^{(k)} \|} $$
   
4. Aproximar el autovalor mediante el cociente de Rayleigh:
   
   $$ \lambda^{(k)} = \frac{(x^{(k)})^T A x^{(k)}}{(x^{(k)})^T x^{(k)}} $$
   
5. Repetir los pasos anteriores hasta que la diferencia entre valores sucesivos de $\lambda$ sea menor que un umbral de tolerancia.

---

*a)* Escriba una función llamada `potencia_inversa` que dada una matriz A encuentre su valor propio menos dominante (con una tolerancia de 1e-7 usando la norma Euclidiana) y su correspondiente autovector. Para obtener $y^{(k)}$ puede emplear `np.linalg.solve` recordando que $Ay = x$.

*b)* Usando la matriz A, encuentre el $\lambda_{min}$ y el correspondiente autovector. Examine la convergencia de su autovalor en función de las iteraciones, esto es, haga un gráfico de $\lambda(k)$

$$
\begin{bmatrix}
5 & 2 \\
2 & 2
\end{bmatrix}
$$

*c)* Compare su resultado con `np.linalg.eig`

In [ ]:
# Aqui va su código

#4

**(10 puntos)**
Sea $A$ una matriz cuadrada de tamaño $n \times n$. Se sabe que el determinante de una matriz es igual al producto de sus autovalores:

$$
\det(A) = \prod_{i=1}^{n} \lambda_i
$$

donde $\lambda_1, \lambda_2, \dots, \lambda_n$ son los autovalores de $A$.

Usando una matriz aleatoria $A$ de $5\times 5$, demuestre esta propiedad. Puede usar las funciones de `np.linalg`.

In [132]:
from numpy import random

A = np.random.randint(10, size=(5, 5))
print(A)

det = np.linalg.det(A)
print(det)

autovalores = np.linalg.eigvals(A)
print(autovalores)
det2 = np.prod(autovalores)
print(det2)

#El valor que se calcula con np.linalg es aproximadamente el valor que se obtiene con el método manual.

[[0 4 7 0 1]
 [0 0 7 8 8]
 [6 2 3 3 3]
 [4 2 9 2 0]
 [1 5 8 6 7]]
2466.0000000000023
[18.93042174+0.j          0.92940826+2.62828221j  0.92940826-2.62828221j
 -2.79744763+0.j         -5.99179063+0.j        ]
(2466.0000000000086-1.1909918711807465e-13j)
